In [1]:
import numpy as np
import pandas as pd
import branca.colormap as cm
from folium.plugins import HeatMap
import folium
from sklearn.svm import OneClassSVM
import flood_tool.analysis as ana

In [2]:
def get_intersect_stations(daily_data, station_data,is_it_rainfall):
    """Get intersect stations of the input dataset and station library.
    
    Parameters
    ----------
    daily_data: dataframe
        Daily rainfall&tide&river data.
    station_data: dataframe
        Station library data from 'stations.csv'(changable).
    is_it_rainfall: str('yes' or 'no')
        Is the input daily_data rainfall or not(river&tide).

    Returns
    -------
    list
        Intersected stationReferences.
    """
    if is_it_rainfall == 'yes':
        stations_all = station_data[station_data['stationName']=='Rainfall station']['stationReference'].unique()
    elif is_it_rainfall == 'no':
        stations_all = station_data[station_data['stationName']!='Rainfall station']['stationReference'].unique()
    daily_station = daily_data['stationReference'].unique()
    return np.intersect1d(stations_all,daily_station)


def replace_string(x):
    """Replace string to float.
    
    Parameters
    ----------
    x: str
        Values that are in string format.
   
    Returns
    -------
    float/np.nan
        Transformed string data.
    """
    try:out = float(x)
    except:out = np.nan
    return out


def river_tide_at_risk_offline(offline_filename):
    """Locations that will be at risk from river and tide data.(Offline dataset)
    
    Parameters
    ----------
    offline_filename: str
        The name of the offline data file.

    Returns
    -------
    dataframe
        Stations that will be at risk.
    """
#   get river and tide data from the station file and drop the rows whose 'typicalRangeHigh' is nan.
    rt_data = pd.read_csv(offline_filename)
    station = pd.read_csv('flood_tool/resources/stations.csv')
    station_rt = station[station.stationName != 'Rainfall station']
    drop_index = station_rt[np.isnan(station_rt['typicalRangeHigh'])].index
    station_rt = station_rt.drop(index=drop_index)

    intersect_stations = get_intersect_stations(rt_data, station, 'no')
    station_rt = station_rt[np.isin(station_rt['stationReference'],intersect_stations)]
    
#   get river and tide data from the input file and drop the rows whose 'value' is nan, not a number or a outlier.
    rt_data = rt_data[(rt_data['qualifier']== 'Stage') | (rt_data['qualifier']== 'Tidal Level')]
    rt_data['value'] = rt_data['value'].apply(replace_string).apply(lambda x: np.nan if x>35 or x<0 else x)
    rt_data = rt_data.drop(index = rt_data[rt_data.value.isnull()].index)
    
#   get the max value of each station.
    rt_data = rt_data[np.isin(rt_data['stationReference'],intersect_stations)]
    rt_max = rt_data.groupby(['stationReference']).agg({'value':'max','unitName':'max'})
    
#   get the stations that will be at risk (whose value is higher than 'typicalRangeHigh').
#   There is 'value' data from some stations whose unit(mm) is not the same as others(m) but we noticed that these stations don't have a record of 'typicalRangeHigh'(and other columns), which means we will drop the data from these stations cuz we don't have a normal value('typicalRangeHigh') to compare with and make a conclusion to say that these locations are at risk.
    cross = pd.merge(rt_max,station_rt,how='left',on=['stationReference'])
    at_risk = cross[cross['value'] > cross['typicalRangeHigh']]
    return at_risk


def river_tide_at_risk_live(live_filename):
    """Locations that will be at risk from river and tide data.(Live dataset)
    
    Parameters
    ----------
    live_filename: str
        The name of the live data file.

    Returns
    -------
    dataframe
        Stations that will be at risk.
    """
    rt_data_live = pd.read_csv(live_filename)
    rt_data_live = rt_data_live[(rt_data_live['stationName'] != 'Rainfall station')] 
    rt_data_live['value'] = rt_data_live['value'].apply(replace_string).apply(lambda x: np.nan if x>35 or x<0 else x)
    rt_data_live = rt_data_live.drop(index = rt_data_live[rt_data_live.value.isnull()].index)
    rt_max_live = rt_data_live.groupby(['stationReference']).agg({'value':'max','latitude':'max','longitude':'max','typicalRangeHigh':'max'})    
    at_risk_live = rt_max_live[rt_max_live['value'] > rt_max_live['typicalRangeHigh']]
    return at_risk_live


def rain_at_risk(new_data_file, typical_data_file, station_data_file, thres=5.0, detect_anomaly=True):
    '''
    Find rainfall stations where there are potential flood risk.
    
    Parameters
    ----------
    new_data_file: str
        File name of the .csv file that contains rainfall station records to estimate risk.
    typical_data_file: str
        File name of the .csv file that contains rainfall station records of a typical day.
    station_data_file: str
        File name of the .csv file that contains basic rainfall station information.
    thres: float, optional
        If rainfall value is above this threshold, then it is regarded as risky.
    detect_anomaly: bool, optional
        If True, then automatically find a threshold using anomaly detection.
        
    Returns
    ----------
    risk_stations: DataFrame
        DataFrame that contains potentially risky rainfall station records.
    '''
    
    # Load data and do necessary data cleaning
    typical_day = pd.read_csv(typical_data_file)
    station = pd.read_csv(station_data_file)
    if 'parameter' in typical_day.columns:
        rain_typical = typical_day[typical_day['parameter']=='rainfall']
    else:
        rain_typical = typical_day[typical_day['stationName']=='Rainfall station']
    
    rain_typical = rain_typical[rain_typical['value'].notna()]
    rain_typical['value'] = pd.to_numeric(rain_typical['value'])
    
    new_data = pd.read_csv(new_data_file)
    if 'parameter' in new_data.columns:
        new_data_rain = new_data[new_data['parameter']=='rainfall']
    else:
        new_data_rain = new_data[new_data['stationName']=='Rainfall station']
    new_data_rain = new_data_rain[new_data_rain['value'].notna()]
    new_data_rain['value'] = pd.to_numeric(new_data_rain['value'])
    
    # Remove standalone data
    intersect_stations = get_intersect_stations(new_data_rain, station, 'yes')
    rain_typical = rain_typical[np.isin(rain_typical['stationReference'], intersect_stations)]
    new_data_rain = new_data_rain[np.isin(new_data_rain['stationReference'], intersect_stations)]
    
    total_new_rain = pd.DataFrame(new_data_rain.groupby('stationReference')['value'].sum())
    
    # Run anomaly detection
    if detect_anomaly:
        thres_anomaly = compare_with_typical_get_thres(total_new_rain, rain_typical, fraction_outlier=0.01)
        thres = max(thres, thres_anomaly)
        
    # Find potentially risky stations using the threshold
    risk_stations = total_new_rain[total_new_rain['value']>thres]
    risk_stations_data = station[station['stationReference'].isin(risk_stations.index.values)]
    risk_stations_data = risk_stations_data.set_index('stationReference')
    risk_stations = risk_stations.join(risk_stations_data, on='stationReference')
    return risk_stations


def compare_with_typical_get_thres(total_new_rain, rain_typical, fraction_outlier=0.05):
    '''
    Compare new data with typical data and run an anomaly detection. 
    New threshold is inferred from detected anomaly data.
    
    Parameters
    ----------
    total_new_rain: DataFrame
        DataFrame containing new data. 
    rain_typical: DataFrame
        DataFrame containing typical data. 
    fraction_outlier: float
        Fraction that data points are considered as anomaly. 
        
    Returns
    ----------
    thres: float
        Smallest rainfall value in anomaly data is selected as the new threshold.
    '''
    
    # Prepare and join two DataFrames, two 'value' columns are used as feature
    total_typical_rain = pd.DataFrame(rain_typical.groupby('stationReference')['value'].sum())
    new_typical_difference = total_new_rain.join(total_typical_rain, on='stationReference', 
                                                 lsuffix='_new', rsuffix='_typical')
    new_typical_difference.dropna(inplace=True)
    
    # Fit anomaly detection model, here we use OneClassSVM
    ocsvm = OneClassSVM(nu=fraction_outlier).fit_predict(new_typical_difference)
    new_typical_difference['anomaly'] = ocsvm
    
    # Find smallest rainfall value that makes data anomaly, which is selected as new threshold
    new_typical_difference.sort_values(by=['value_new', 'value_typical'], ascending=[False, False], inplace=True)
    record = new_typical_difference[new_typical_difference['anomaly']==-1].iloc[-1]
    thres = 0.5 * (record['value_new'] + record['value_typical'])
    return thres

In [3]:
at_risk_tide_river_offline = river_tide_at_risk_offline('flood_tool/resources/typical_day.csv')
at_risk_tide_river_live = river_tide_at_risk_live('2022-11-24T15-45-00Z.csv')

In [4]:
at_risk_rainfall = rain_at_risk('2022-11-24T15-45-00Z.csv', 'flood_tool/resources/typical_day.csv', 
                       'flood_tool/resources/stations.csv', thres=4.0)

In [5]:
ana.plot_84_map(at_risk_tide_river_offline['latitude'].values,at_risk_tide_river_offline['longitude'].values,at_risk_tide_river_offline['value'].values,'tide_river_offline')

In [6]:
ana.plot_84_map(at_risk_tide_river_live ['latitude'].values,at_risk_tide_river_live ['longitude'].values,at_risk_tide_river_live['value'].values,'tide_river_live')

In [7]:
ana.plot_84_map(at_risk_rainfall['latitude'].values,at_risk_rainfall['longitude'].values,at_risk_rainfall['value'].values,'rainfall')